In [1]:
%%capture

import audata as aud
import h5py as h5
from tsfresh import extract_features
import pandas as pd
import numpy as np
import scipy.stats
import concurrent.futures
import threading
import traceback
import biosignalsnotebooks as bsnb
from multiprocessing import Pool, TimeoutError
from sklearn.linear_model import LinearRegression

import rpy2
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr
robjects.r['source']('pf.R')
robjects.r['source']('feature_extract_alert_p3.R')

files_dir = '/home/auvdata/projects/conditionc-new/originals/'

In [2]:
labels = pd.read_csv('../finalLabels.csv')

# Filter alerts with less than 3 minute timespan
labels['time'] = labels['right'] - labels['left']
before = labels.shape[0]
labels = labels[labels['time'] > 180]
labels.drop(columns=['time'], inplace=True)
print(f"Removed {before-labels.shape[0]} labels with less than a 3-minute timespan.")

Removed 80 labels with less than a 3-minute timespan.


# Function Definitions

In [59]:
def run(featurize, featurizeThese, columns):
    
    featurizedData = pd.DataFrame(columns=columns)

    i = 0
    print("Featurizing (sync)...\n")
    for row in featurizeThese.iterrows():
        i += 1
        print(f"{i} of {featurizeThese.shape[0]}")
        newRow = featurize(row)
        if newRow is not None:
            featurizedData = featurizedData.append(pd.DataFrame([newRow], columns=featurizedData.columns), ignore_index=True)
        else:
            print("None returned. Skipping!")
                
    return finalizeDataset(featurizedData)

def run_async(featurize, featurizeThese, columns, numProcs):
    
    featurizedData = pd.DataFrame(columns=columns)

    with Pool(numProcs) as p:

        i = 0
        num = len(featurizeThese)

        print("Featurizing (async)...\n")
        it = p.imap_unordered(featurize, featurizeThese.iterrows())
        while i < num:

            i += 1

            try:
                #newRow = it.next(timeout=600)
                newRow = it.next()
                print(f"{i} of {num} returned.")
                print("New row:", newRow, sep="\n")
                if newRow is not None:
                    featurizedData = featurizedData.append(pd.DataFrame([newRow], columns=featurizedData.columns), ignore_index=True)
                else:
                    print("None returned. Skipping!")
            except TimeoutError as te:
                print(f"Timed out. {te}")
                traceback.print_exc()
            except StopIteration:
                print("Pool iteration ended.")
                break
            except Exception as e:
                print(f"Other exception... {e}")
                traceback.print_exc()
            
    return finalizeDataset(featurizedData)
            
def finalizeDataset(ds):

    # Replace any inf values with nan
    return ds.replace([np.inf, -np.inf], np.nan)

# Open the file in both audata and h5py
def openFiles(fn):
    return aud.File.open(files_dir+fn, readonly=True), h5.File(files_dir+fn, 'r')
    
# Validate that data is at least minSamplesToBeValid, not a 0 timespan
def validateDataset(data):
    return False if data.shape[0] < minSamplesToBeValid or data.iloc[-1].time - data.iloc[0].time == 0 else True
   
# Convert a list/vector(?) response from R into a dict
def getDictFromR(o):
    return dict(zip(o.names, map(list,list(o))))

def isWaveform(s):
    return s.startswith('/data/waveforms')

# Return basic statistics for a pandas series
def getBasicStats(ds):
    
    features = []
    
    _mean = ds.mean()
    _median = ds.median()
    _std = ds.std()
    _min = ds.min()
    _max = ds.max()
    
    features.append(_mean) # mean
    features.append(_std) # sd
    features.append(_std/_mean if _mean != 0 else np.nan)# cv
    features.append((ds-_mean).median()) # mad
    features.append(hrrr.shape[0]) # n
    features.append(_min) # min
    features.append(_max) # max
    features.append(_median) # median
    features.append(_max-_min) # range
    features.append((_max-_min)/_median if _median != 0 else np.nan)# range_ratio
    
    return features

# Get best numeric series from available options
# f=file, so=series options, left=left timestamp
def getBestSeries(f, h5f, so, left):
    
    printout and print(f"getBestSeries(f, h5f, {so}, {left}) started.")
    
    finalSeries = None
    finalSeriesSampleCount = -1
    finalSeriesTREF = None
    finalSeriesDF = None
    finalSeriesData = None

    # Iterate through the series options
    for s in so:

        # If the series is not available in the file, skip
        if s not in f:
            continue
            
        tref = f[s].time_reference.timestamp()

        if isWaveform(s):
            
            df = h5f[s]
            data = pd.DataFrame(df[(df['time'] > left-tref) & (df['time'] < left+60*3-tref)])
        
        else:
            
            df = pd.DataFrame(f[s].get(raw=True)).drop_duplicates(subset=['time'], ignore_index=True)

            # Downsample if requested. Note we do not downsample NBP because it is already a low-rate measurement.
            if downsample_to_ppinnc1 and sk in ['hr', 'rr', 'spo2']:
                df = df.iloc[::20].reset_index(drop=True)

            data = df[ (df.time > left-tref) & (df.time < left+60*3-tref) ]

        # If data validates and has more samples than current final series, it's our new final series
        if validateDataset(data) and data.shape[0] > finalSeriesSampleCount:

            finalSeries = s
            finalSeriesSampleCount = data.shape[0]
            finalSeriesTREF = tref
            finalSeriesDF = df
            finalSeriesData = data
            
    printout and print(f"getBestSeries(f, h5f, {so}, {left}) returning.")

    return finalSeries, finalSeriesTREF, finalSeriesDF, finalSeriesData

def featurize(row):

    # Grab the row content
    row = row[1]
    
    printout and print(f"File: {row['filename']}\nAlert Series: {row['series']}\nLabel: {'REAL' if row['real_vs_art']==1 else 'ARTIFACT'}")

    # Open the file in both audata and h5py
    (f, h5f) = openFiles(row['filename'])
    
    # This will hold this alert's computed features
    features = [row['filename'], row['series'], row['left'], row['right'], row['real_vs_art']]
    
    # Iterate through the signal types (each signal type e.g. hr, spo2 will get one set of features)
    for sk, so in seriesOptions.items():
        
        # TODO(gus): If this sk is for the label target series category, use the label target series.
        
        (series, tref, df, data) = getBestSeries(f, h5f, so, row['left'])
        
        # If we have no available series for this signal type, add empty values for the features and move on.
        if series is None:
            print(f"No data available for {sk}!")
            
            # TODO(gus): temp
            if sk == 'pleth' or sk == 'ecgii':
                # this is not temp, just pull it out of the if
                features = features + ( [np.nan]*(len(featuresList) + (3 if sk == 'pleth' else 0)) )
            
            # If we have no data for our CRI target series (e.g. if we've invalidated it), skip this CRI alert entirely
            if row['series'].split(':')[0] in seriesOptions[sk]:
                print(f"Not enough / invalid data for for CRI target series. Discluding this CRI alert!")
                return None
            
            continue
        
        # TODO(gus): temp
        if sk != 'pleth' and sk != 'ecgii':
            print(f"Temporarily skipping {sk}")
            continue
        
        printout and print(f"Featurizing series {series}, timespan: {round((data.iloc[-1].time - data.iloc[0].time) / 60, 2)} minutes, sample count: {data.shape[0]}")
        
        _mean = data.value.mean()
        _median = data.value.median()
        _std = data.value.std()
        _min = data.value.min()
        _max = data.value.max()
        features.append(_mean) # mean
        features.append(_std) # sd
        features.append(_std/_mean if _mean != 0 else np.nan) # cv
        features.append((data.value - _mean).median()) # mad
        features.append(data.shape[0]) # n
        features.append(_min) # min
        features.append(_max) # max
        features.append(_median) # median
        features.append(_max-_min) # range
        features.append((_max-_min)/_median if _median != 0 else np.nan) # range_ratio
        
        printout and print("Retrieving trail.")
        trail = df[ (df['time'] < row['left']-tref) & (df['time'] > row['left']-tref-15*60) ]
        if isWaveform(series):
            trail = pd.DataFrame(trail)
        printout and print("Retrieving trail complete.")

        features.append( (data.shape[0])/(row['right']-row['left']) ) # data_den. note: use cri timespan, not sample timespan

        trail_count = trail.shape[0]
        if not validateDataset(trail):
            features = features + ( [np.nan]*2 ) # data_den_trail, data_den_trail2
        else:
            features.append(trail_count / (trail.time.iloc[-1]-trail.time.iloc[0])) # data_den_trail
            features.append(trail_count / (15*60)) # data_den_trail2

        # TODO(gus): Ignoring this for waveforms for now
        if isWaveform(series):
            features.append(np.nan) # delta_t
        else:
            features.append(df.loc[data.index[0], 'time'] - df.loc[data.index[0]-1, 'time'] if data.index[0] > 0 else np.nan) # delta_t
        features.append(data.time.diff().max()) # max_gap

        # skipping for now... spec_ratio
        # skipping for now... max_spec
#         with localconverter(robjects.default_converter + pandas2ri.converter):
#             ret = getDictFromR(robjects.r['get_spectral_lomb'](data['time'], data['value']))
#         print('spec:', ret)

        printout and print("Retrieving dl4.")
        dl4 = df[(df['time'] >= row['left']-tref-4*60) & (df['time'] < row['left']-tref)]
        if isWaveform(series):
            dl4 = pd.DataFrame(dl4)
        printout and print("Retrieving dl4 complete.")

        if not validateDataset(dl4):

            features = features + ( [np.nan]*10 ) # delta_mean, delta_std, MWW-test statistic & p-value, KS-test statistic & p-value, t-test statistic & p-value, f-test statistic & p-value

        else:

            features.append(abs(dl4.value.mean()-_mean)) # delta_mean
            features.append(abs(dl4.value.std()-_std)) # delta_std

            #print(data['value'], dl4['value'])
            try:
                (mww_stat, mww_pval) = scipy.stats.mannwhitneyu(data['value'], dl4['value'])
                features.append(mww_stat)# MWW-test statistic
                features.append(mww_pval)# MWW-test p-value
            except:
                features = features + ( [np.nan]*2 )

            (ks_stat, ks_pval) = scipy.stats.ks_2samp(data['value'], dl4['value'])
            features.append(ks_stat) # KS-test statistic
            features.append(ks_pval) # KS-test p-value

            (tt_stat, tt_pval) = scipy.stats.ttest_ind(data['value'], dl4['value'])
            features.append(tt_stat) # t-test statistic
            features.append(tt_pval) # t-test p-value

            try:
                with localconverter(robjects.default_converter + pandas2ri.converter):
                    ret = getDictFromR(robjects.r['var.test'](data['value'], dl4['value']))
                features.append(ret['statistic'][0]) # f-test statistic
                features.append(ret['p.value'][0]) # f-test p-value
            except Exception:
                traceback.print_exc()
                print(row, data['value'], dl4['value'], sep="\n")
                features.append(np.nan)
                features.append(np.nan)

        # this section calculates f-test in python, but results often differ from r
#         x = data['value']
#         y = dl4['value']
#         f = x.var()/y.var()
#         p = 2*scipy.stats.f.cdf(f, x.shape[0]-1, y.shape[0]-1)
#         print(f, p)
#         print('p-python/p-r', vtr['p.value'][0]/p)

        slrm = LinearRegression()
        slrm.fit(data[['time']], data['value'])
        features.append(slrm.coef_[0]) # slope

        with localconverter(robjects.default_converter + pandas2ri.converter):
            ret = robjects.r['get_robust_slope'](data['time'], data['value'])
        features.append(ret[0]) # rslope

        with localconverter(robjects.default_converter + pandas2ri.converter):
            ret = robjects.r['get_slope_break'](data['time'], data['value'])
        features.append(ret[0][0]) # slope_before
        features.append(ret[1][0]) # slope_after
        features.append(ret[2][0]) # num_breakpoint

        data_dropduptimes = data.drop_duplicates(subset=['time'], keep='last')
        dydx = (data_dropduptimes.value.diff() / data_dropduptimes.time.diff())[1:]
        features.append(dydx.max()) # diff1_max
        features.append(dydx.min()) # diff1_min

        # TODO(gus): Ignoring this for waveforms for now
        if isWaveform(series):
            features = features + ([np.nan]*2)
        else:
            with localconverter(robjects.default_converter + pandas2ri.converter):
                ret = getDictFromR(robjects.r['comp_grads'](data['time'], data['value']))
            features.append(ret['max_grad'][0]) # max_grad
            features.append(ret['min_grad'][0]) # min_grad

        with localconverter(robjects.default_converter + pandas2ri.converter):
            ret = getDictFromR(robjects.r['get_rsq'](data['time'], data['value']))
        features.append(ret['rsq'][0]) # quad_rsq
        features.append(ret['coef1'][0]) # quad_coef1
        features.append(ret['coef2'][0]) # quad_coef2
        features.append(ret['var.res'][0]) # quad_resvar

        with localconverter(robjects.default_converter + pandas2ri.converter):
            ret = getDictFromR(robjects.r['get.osci.index'](data['time'], data['value']))
        features.append(ret['up'][0]) # osi_up
        features.append(ret['down'][0]) # osi_down
        features.append(ret['ratio'][0]) # osi_ratio
        
        if sk == 'pleth':
            
            data['time'] = pd.to_datetime(data['time']+tref, unit='s')
            data.set_index('time', inplace=True)
            features.append(data.resample('3s').agg(lambda x: ((x-x.mean()/x.std())**3).sum()/x.shape[0]).mean()[0]) # skew
            features.append(data.resample('3s').agg(lambda x: ((x-x.mean()/x.std())**4).sum()/x.shape[0]).mean()[0]) # kurtosis
            features.append(data.resample('3s').agg(lambda x: (x**2 * np.log(x**2)).sum()*-1).mean()[0]) # entropy

    print("File done.")
    printout and print()
    
    return features

def featurize_targeted(row):
    
    # Grab the row content
    row = row[1]
    
    # Open the file in both audata and h5py
    (f, h5f) = openFiles(row['filename'])
    
    # This will hold this alert's computed features
    features = [row['filename'], row['series'], row['left'], row['right'], row['label']]
    
    sk = 'ecgii'
    
    (series, tref, df, data) = getBestSeries(f, h5f, seriesOptions[sk], row['left'])
    
    # If we have no available series for this signal type, add empty values for the features and move on.
    if series is None:
        
        print(f"No data available for {sk}!")
        features = features + [np.nan]
        
    else:
        
        try:
        
            freq = int(round(1/(data.loc[1, 'time']-data.loc[0, 'time'])))

            time_r_peaks, amplitude_r_peaks = bsnb.detect_r_peaks(data['value'], freq, time_units=True, plot_result=False)

            vpp_signal_ecg = np.ptp(data['value'])

            # For this task, we will follow the same procedure as shown before, but store the values in a list, so that we can then calculate the mean value.
            vpp_noise_ecg = []
            for t in time_r_peaks:
                start = int((t + 0.5) * freq) # 0.5 - time between a peak and a flat 
                end = int((t + 0.65)* freq) # 0.65 time between a peak and the end of the flat
                interval = data['value'][start:end]
                if interval.shape[0] == 0:
                    print("Skipping len=0 noise segment.")
                    continue
                vpp = np.ptp(interval)
                vpp_noise_ecg.append(vpp)

            vpp_noise_ecg = np.mean(vpp_noise_ecg)

            features.append(vpp_signal_ecg/vpp_noise_ecg)
            
        except Exception as e:
            
            print(f"Exception occurred while computing SNR: {e}")
            features.append(np.nan)
        
    (seriesHR, trefHR, dfHR, dataHR) = getBestSeries(f, h5f, seriesOptions['hr'], row['left'])
    (seriesRR, trefRR, dfRR, dataRR) = getBestSeries(f, h5f, seriesOptions['rr'], row['left'])
    if seriesHR is None or seriesRR is None:
        print(f"No data available for hr+rr!")
        features = features + [np.nan]*10
        
    else:
        
        dataHR.set_index('time', inplace=True)
        dataRR.set_index('time', inplace=True)
        hrrr = pd.concat([dataHR, dataRR], axis=1).dropna()
        hrrr.columns = ['hr', 'rr']
        features = features + getBasicStats(hrrr['hr'] / hrrr['rr'])

    return features

# Action

In [62]:
downsample_to_ppinnc1 = False
printout = False
minSamplesToBeValid = 5

# Standard list of features to compute for each series
featuresList = [
    'mean', 'sd', 'cv', 'mad', 'n', 'min', 'max', 'median', 'range', 'range_ratio',
    'data_den', 'data_den_trail', 'data_den_trail2', 'delta_t', 'max_gap',
    # 'spec_ratio', 'max_spec',
    'delta_mean', 'delta_sd', 'MWW_stat', 'MWW_pvalue', 'KS_stat', 'KS_pvalue', 't_stat', 't_pvalue', 'F_stat', 'F_pvalue',
    'slope', 'rslope', 'slope_before', 'slope_after', 'num_breakpoint', 'diff1_max', 'diff1_min', 'max_grad', 'min_grad', 'quad_rsq', 'quad_coef1', 'quad_coef2', 'quad_resvar', 'osi_up', 'osi_down', 'osi_ratio',
]

# List of series to compute features for, by signal type
seriesOptions = {
    'hr':    ['/data/numerics/HR.HR'],
    'rr':    ['/data/numerics/RR.RR'],
    'spo2':  ['/data/numerics/SpO₂.SpO₂', '/data/numerics/SpO₂T.SpO₂T'],
    'bpd':   ['/data/numerics/NBP.NBPd', '/data/numerics/NBP.NBP-D'],
    'bps':   ['/data/numerics/NBP.NBPs', '/data/numerics/NBP.NBP-S'],
}
if not downsample_to_ppinnc1:
    seriesOptions['abpd'] = ['/data/numerics/AR1-D', '/data/numerics/ART.Diastolic']
    seriesOptions['abps'] = ['/data/numerics/AR1-S', '/data/numerics/ART.Systolic']
    seriesOptions['ecgii'] = ['/data/waveforms/II']
    seriesOptions['pleth'] = ['/data/waveforms/Pleth', '/data/waveforms/PlethT']

# The columns end up being filename, series, left, right, label, and then one set of featuresList column names for each series.
columns = \
    ['filename', 'series', 'left', 'right', 'label']+ \
    [ser +'_' + feat for ser in ['hr', 'rr', 'spo2', 'bpd', 'bps'] for feat in featuresList]+ \
    ([ser +'_' + feat for ser in ['abpd', 'abps'] for feat in featuresList] if not downsample_to_ppinnc1 else [])+ \
    ['ecgii_' + feat for feat in featuresList]+ \
    ['pleth_' + feat for feat in featuresList+['skew', 'kurtosis', 'entropy']]
    
# Label set to featurize
# k=0; featurizeThese = labels.iloc[k:k+1]
# featurizeThese = labels[20:45]
# featurizeThese = labels.drop(labels.index[97])[95:]
featurizeThese = labels

# run(featurize, featurizeThese, columns)


# Label set to featurize
prevFeatureSet = pd.read_csv('data_full_pleth2.csv')[317:318]
columns = \
    ['filename', 'series', 'left', 'right', 'label', 'ecgii_snr']+ \
    ['hrrr_ratio_'+feat for feat in ['mean', 'sd', 'cv', 'mad', 'n', 'min', 'max', 'median', 'range', 'range_ratio']]
# featurizedData = run(featurize_targeted, prevFeatureSet, columns)
featurizedData = run_async(featurize_targeted, prevFeatureSet, columns, 18)




print("Run complete.")

with pd.option_context("display.max_columns", 1000):
    print("Featurized Data:")
    display(featurizedData)

Featurizing (async)...

Exception occurred while computing SNR: The length of the input vector x must be greater than padlen, which is 150.
No data available for hr+rr!
1 of 1 returned.
New row:
['20190805_1157669_1498169.h5', '/data/numerics/HR.HR:value', 1539680314, 1539681809, 0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
Run complete.
Featurized Data:


,filename,series,left,right,label,ecgii_snr,hrrr_ratio_mean,hrrr_ratio_sd,hrrr_ratio_cv,hrrr_ratio_mad,hrrr_ratio_n,hrrr_ratio_min,hrrr_ratio_max,hrrr_ratio_median,hrrr_ratio_range,hrrr_ratio_range_ratio
0,20190805_1157669_1498169.h5,/data/numerics/HR.HR:value,1539680314,1539681809,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
labels

In [ ]:
prevFeatureSet

# Export

In [63]:
featurizedDataMissingOneSafe = featurizedDataMissingOne.copy()

In [ ]:
# ### Export to model-specific datasets

# suf = '_pleth'

# ds = featurizedData_PLETH

# ds.to_csv(f'data_full{suf}.csv', index=False)
# ds.drop(columns=['series', 'left', 'right']).to_csv(f'data_withfn{suf}.csv', index=False)

# ds[ds['series'] == '/data/numerics/HR.HR:value'].drop(columns=['series', 'left', 'right']).to_csv(f'data_hr_withfn{suf}.csv', index=False)
# ds[ds['series'] == '/data/numerics/RR.RR:value'].drop(columns=['series', 'left', 'right']).to_csv(f'data_rr_withfn{suf}.csv', index=False)
# ds[ds['series'].isin(['/data/numerics/SpO₂.SpO₂:value', '/data/numerics/SpO₂T.SpO₂T:value'])].drop(columns=['series', 'left', 'right']).to_csv(f'data_spo2_withfn{suf}.csv', index=False)

In [44]:
featurizedData.sort_values(by=['filename', 'series', 'left'])

,filename,series,left,right,label,ecgii_snr,hrrr_ratio_mean,hrrr_ratio_sd,hrrr_ratio_cv,hrrr_ratio_mad,hrrr_ratio_n,hrrr_ratio_min,hrrr_ratio_max,hrrr_ratio_median,hrrr_ratio_range,hrrr_ratio_range_ratio
16,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1515595508,1515596030,1,14.909494,8.677217,1.638820,0.188865,0.197783,141.0,4.800000,11.916667,8.875000,7.116667,0.801878
17,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1515638422,1515643674,1,14.332886,7.477863,1.204421,0.161065,-0.007275,141.0,5.111111,10.538462,7.470588,5.427350,0.726496
9,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1515912676,1515930671,1,39.233044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1516035392,1516037744,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1516039795,1516052674,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,20200630_1934279_1011611.h5,/data/numerics/HR.HR:value,1535443106,1535443609,1,1.945634,5.836121,0.431140,0.073874,0.063879,141.0,4.640000,6.823529,5.900000,2.183529,0.370090
589,20200630_1934279_1011611.h5,/data/numerics/RR.RR:value,1535421926,1535422231,1,17.361591,2.456410,0.502780,0.204681,-0.087632,141.0,1.652174,3.545455,2.368778,1.893281,0.799265
590,20200630_1934279_1011611.h5,/data/numerics/RR.RR:value,1535597897,1535599266,1,32.575806,4.291524,0.379917,0.088527,-0.041524,141.0,3.432432,5.565217,4.250000,2.132785,0.501832
592,20200630_1934279_1011611.h5,/data/numerics/SpO₂.SpO₂:value,1535465918,1535466465,1,29.606163,3.319307,0.085594,0.025787,0.014026,141.0,3.136364,3.500000,3.333333,0.363636,0.109091


In [42]:
prevFeatureSet.sort_values(by=['filename', 'series', 'left'])

,filename,series,left,right,label,hr_mean,hr_sd,hr_cv,hr_mad,hr_n,...,pleth_mad,pleth_n,pleth_min,pleth_max,pleth_median,pleth_range,pleth_range_ratio,pleth_skew,pleth_kurtosis,pleth_entropy
0,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1515595508,1515596030,1,139.865248,3.468676,0.024800,1.134752,141.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1515638422,1515643674,1,138.085106,3.485663,0.025243,-0.085106,141.0,...,-0.001627,22500.0,0.000000,1.000000,0.492796,1.000000,2.029237,-859.174011,14581.756836,116.537605
2,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1515912676,1515930671,1,63.564286,3.393203,0.053382,0.435714,140.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1516035392,1516037744,1,111.957447,5.856464,0.052310,-0.957447,141.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1516039795,1516052674,1,161.765957,1.830840,0.011318,0.234043,141.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,20200630_1934279_1011611.h5,/data/numerics/HR.HR:value,1535443106,1535443609,1,115.426136,2.403616,0.020824,-0.426136,176.0,...,0.014405,22500.0,0.250305,0.750183,0.514774,0.499878,0.971063,-39.487469,136.033524,118.272507
596,20200630_1934279_1011611.h5,/data/numerics/RR.RR:value,1535421926,1535422231,1,79.392045,3.450837,0.043466,-0.392045,176.0,...,0.004972,22500.0,0.250305,0.750183,0.505006,0.499878,0.989845,-29.736944,93.264023,116.483162
597,20200630_1934279_1011611.h5,/data/numerics/RR.RR:value,1535597897,1535599266,1,127.142045,1.236236,0.009723,-0.142045,176.0,...,0.007035,22500.0,0.250305,0.750183,0.505739,0.499878,0.988411,-51.701866,200.224915,119.124802
598,20200630_1934279_1011611.h5,/data/numerics/SpO₂.SpO₂:value,1535465918,1535466465,1,69.863636,0.712048,0.010192,0.136364,176.0,...,-0.014382,22500.0,0.250305,0.750183,0.485958,0.499878,1.028643,-35.061020,115.517738,117.092415


In [46]:
a=featurizedData.sort_values(by=['filename', 'series', 'left']).reset_index()
b=prevFeatureSet.sort_values(by=['filename', 'series', 'left']).reset_index()

In [47]:
a[~a.isin(b).all(1)]

,index,filename,series,left,right,label,ecgii_snr,hrrr_ratio_mean,hrrr_ratio_sd,hrrr_ratio_cv,hrrr_ratio_mad,hrrr_ratio_n,hrrr_ratio_min,hrrr_ratio_max,hrrr_ratio_median,hrrr_ratio_range,hrrr_ratio_range_ratio
0,16,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1515595508,1515596030,1,14.909494,8.677217,1.638820,0.188865,0.197783,141.0,4.800000,11.916667,8.875000,7.116667,0.801878
1,17,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1515638422,1515643674,1,14.332886,7.477863,1.204421,0.161065,-0.007275,141.0,5.111111,10.538462,7.470588,5.427350,0.726496
2,9,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1515912676,1515930671,1,39.233044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1516035392,1516037744,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1516039795,1516052674,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,591,20200630_1934279_1011611.h5,/data/numerics/HR.HR:value,1535443106,1535443609,1,1.945634,5.836121,0.431140,0.073874,0.063879,141.0,4.640000,6.823529,5.900000,2.183529,0.370090
595,589,20200630_1934279_1011611.h5,/data/numerics/RR.RR:value,1535421926,1535422231,1,17.361591,2.456410,0.502780,0.204681,-0.087632,141.0,1.652174,3.545455,2.368778,1.893281,0.799265
596,590,20200630_1934279_1011611.h5,/data/numerics/RR.RR:value,1535597897,1535599266,1,32.575806,4.291524,0.379917,0.088527,-0.041524,141.0,3.432432,5.565217,4.250000,2.132785,0.501832
597,592,20200630_1934279_1011611.h5,/data/numerics/SpO₂.SpO₂:value,1535465918,1535466465,1,29.606163,3.319307,0.085594,0.025787,0.014026,141.0,3.136364,3.500000,3.333333,0.363636,0.109091


In [48]:
a=a[['filename', 'series', 'left']]
b=b[['filename', 'series', 'left']]

In [52]:
a.values.difference(b.values)

AttributeError: 'numpy.ndarray' object has no attribute 'difference'

In [53]:
ds1 = set([tuple(line) for line in a.values])
ds2 = set([tuple(line) for line in b.values])

In [55]:
ds2.difference(ds1)

{('20190805_1157669_1498169.h5', '/data/numerics/HR.HR:value', 1539680314)}

In [69]:
pd.concat([featurizedDataMissingOne, featurizedData], ignore_index=True).copy()[['filename', 'series', 'left', 'right']].drop_duplicates()

,filename,series,left,right
0,20190523_1070382_1193704.h5,/data/numerics/HR.HR:value,1501725628,1501726167
1,20190523_1070382_1193704.h5,/data/numerics/HR.HR:value,1501605430,1501612529
2,20190523_1070382_1193704.h5,/data/numerics/HR.HR:value,1501660023,1501660527
3,20190523_1070382_1193704.h5,/data/numerics/SpO₂T.SpO₂T:value,1501591667,1501592233
4,20190523_1070382_1193704.h5,/data/numerics/HR.HR:value,1501698845,1501705251
...,...,...,...,...
595,20191112_1903642_1927237.h5,/data/numerics/SpO₂.SpO₂:value,1530421160,1530421684
596,20191112_1903642_1927237.h5,/data/numerics/SpO₂.SpO₂:value,1530390852,1530391282
597,20191112_1903642_1927237.h5,/data/numerics/SpO₂.SpO₂:value,1530316252,1530316656
598,20191112_1903642_1927237.h5,/data/numerics/SpO₂.SpO₂:value,1530512687,1530513128


In [89]:
for f in list(nd.columns):
    print(f)

ecgii_snr
hrrr_ratio_mean
hrrr_ratio_sd
hrrr_ratio_cv
hrrr_ratio_mad
hrrr_ratio_n
hrrr_ratio_min
hrrr_ratio_max
hrrr_ratio_median
hrrr_ratio_range
hrrr_ratio_range_ratio


In [72]:
### Combine datasets (for targeted featurization)

# new data
#nd = featurizedData.copy()
nd = pd.concat([featurizedDataMissingOne, featurizedData], ignore_index=True).copy()
nd.set_index(['filename', 'series', 'left', 'right'], inplace=True)
nd.sort_index(inplace=True)
nd.drop(columns=['label'], inplace=True)

# old data
#od = pd.read_csv('data_full_pleth15s.csv')
#od = prevFeatureSet.copy()
od = pd.read_csv('data_full_pleth2.csv')
od.set_index(['filename', 'series', 'left', 'right'], inplace=True)
od.sort_index(inplace=True)
#od.drop(columns=['pleth_skew', 'pleth_kurtosis', 'pleth_entropy'], inplace=True)

combined = pd.concat([od, nd], axis=1).reset_index()

In [78]:
with pd.option_context("display.max_columns", 1000):
    display(combined)

,filename,series,left,right,label,hr_mean,hr_sd,hr_cv,hr_mad,hr_n,hr_min,hr_max,hr_median,hr_range,hr_range_ratio,hr_data_den,hr_data_den_trail,hr_data_den_trail2,hr_delta_t,hr_max_gap,hr_delta_mean,hr_delta_sd,hr_MWW_stat,hr_MWW_pvalue,hr_KS_stat,hr_KS_pvalue,hr_t_stat,hr_t_pvalue,hr_F_stat,hr_F_pvalue,hr_slope,hr_rslope,hr_slope_before,hr_slope_after,hr_num_breakpoint,hr_diff1_max,hr_diff1_min,hr_max_grad,hr_min_grad,hr_quad_rsq,hr_quad_coef1,hr_quad_coef2,hr_quad_resvar,hr_osi_up,hr_osi_down,hr_osi_ratio,rr_mean,rr_sd,rr_cv,rr_mad,rr_n,rr_min,rr_max,rr_median,rr_range,rr_range_ratio,rr_data_den,rr_data_den_trail,rr_data_den_trail2,rr_delta_t,rr_max_gap,rr_delta_mean,rr_delta_sd,rr_MWW_stat,rr_MWW_pvalue,rr_KS_stat,rr_KS_pvalue,rr_t_stat,rr_t_pvalue,rr_F_stat,rr_F_pvalue,rr_slope,rr_rslope,rr_slope_before,rr_slope_after,rr_num_breakpoint,rr_diff1_max,rr_diff1_min,rr_max_grad,rr_min_grad,rr_quad_rsq,rr_quad_coef1,rr_quad_coef2,rr_quad_resvar,rr_osi_up,rr_osi_down,rr_osi_ratio,spo2_mean,spo2_sd,spo2_cv,spo2_mad,spo2_n,spo2_min,spo2_max,spo2_median,spo2_range,spo2_range_ratio,spo2_data_den,spo2_data_den_trail,spo2_data_den_trail2,spo2_delta_t,spo2_max_gap,spo2_delta_mean,spo2_delta_sd,spo2_MWW_stat,spo2_MWW_pvalue,spo2_KS_stat,spo2_KS_pvalue,spo2_t_stat,spo2_t_pvalue,spo2_F_stat,spo2_F_pvalue,spo2_slope,spo2_rslope,spo2_slope_before,spo2_slope_after,spo2_num_breakpoint,spo2_diff1_max,spo2_diff1_min,spo2_max_grad,spo2_min_grad,spo2_quad_rsq,spo2_quad_coef1,spo2_quad_coef2,spo2_quad_resvar,spo2_osi_up,spo2_osi_down,spo2_osi_ratio,bpd_mean,bpd_sd,bpd_cv,bpd_mad,bpd_n,bpd_min,bpd_max,bpd_median,bpd_range,bpd_range_ratio,bpd_data_den,bpd_data_den_trail,bpd_data_den_trail2,bpd_delta_t,bpd_max_gap,bpd_delta_mean,bpd_delta_sd,bpd_MWW_stat,bpd_MWW_pvalue,bpd_KS_stat,bpd_KS_pvalue,bpd_t_stat,bpd_t_pvalue,bpd_F_stat,bpd_F_pvalue,bpd_slope,bpd_rslope,bpd_slope_before,bpd_slope_after,bpd_num_breakpoint,bpd_diff1_max,bpd_diff1_min,bpd_max_grad,bpd_min_grad,bpd_quad_rsq,bpd_quad_coef1,bpd_quad_coef2,bpd_quad_resvar,bpd_osi_up,bpd_osi_down,bpd_osi_ratio,bps_mean,bps_sd,bps_cv,bps_mad,bps_n,bps_min,bps_max,bps_median,bps_range,bps_range_ratio,bps_data_den,bps_data_den_trail,bps_data_den_trail2,bps_delta_t,bps_max_gap,bps_delta_mean,bps_delta_sd,bps_MWW_stat,bps_MWW_pvalue,bps_KS_stat,bps_KS_pvalue,bps_t_stat,bps_t_pvalue,bps_F_stat,bps_F_pvalue,bps_slope,bps_rslope,bps_slope_before,bps_slope_after,bps_num_breakpoint,bps_diff1_max,bps_diff1_min,bps_max_grad,bps_min_grad,bps_quad_rsq,bps_quad_coef1,bps_quad_coef2,bps_quad_resvar,bps_osi_up,bps_osi_down,bps_osi_ratio,ecgii_mean,ecgii_sd,ecgii_cv,ecgii_mad,ecgii_n,ecgii_min,ecgii_max,ecgii_median,ecgii_range,ecgii_range_ratio,pleth_mean,pleth_sd,pleth_cv,pleth_mad,pleth_n,pleth_min,pleth_max,pleth_median,pleth_range,pleth_range_ratio,pleth_skew,pleth_kurtosis,pleth_entropy,ecgii_snr,hrrr_ratio_mean,hrrr_ratio_sd,hrrr_ratio_cv,hrrr_ratio_mad,hrrr_ratio_n,hrrr_ratio_min,hrrr_ratio_max,hrrr_ratio_median,hrrr_ratio_range,hrrr_ratio_range_ratio
0,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1515595508,1515596030,1,139.865248,3.468676,0.024800,1.134752,141.0,126.0,146.0,141.0,20.0,0.141844,0.270115,0.782809,0.781111,2.048,2.048,52.464179,0.724842,0.0,1.161034e-54,1.000000,0.000000e+00,120.650057,1.440056e-272,0.684180,1.830132e-02,-0.011264,-0.016833,-0.068257,-0.066920,1.0,9.765626,-1.953125,6.616793,-1.135976,0.028826,1.934979e+03,-0.011264,11.684885,0.0,0.0,NaN,16.865248,4.144221,0.245725,-0.865248,141.0,12.0,30.0,16.0,18.0,1.125000,0.270115,0.782809,0.781111,2.048,2.048,1.501612,1.345150,10750.5,1.971148e-03,0.222589,5.499935e-04,3.911740,1.115378e-04,2.192088,6.087411e-07,-0.059093,-0.051371,-0.204129,-0.020814,1.0,5.859375,-2.929688,1.621756,-0.939216,0.555763,9.434094e+03,-0.059093,7.629583,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [83]:
with pd.option_context("display.max_columns", 1000):
    #display(combined.drop(columns=[pre+'_'+feat for pre in ['pleth', 'ecgii'] for feat in ['mean', 'sd', 'cv', 'mad', 'n', 'min', 'max', 'median', 'range', 'range_ratio']]))
    display(combined.drop(columns=[pre+'_'+feat for pre in ['bps', 'bpd'] for feat in featuresList]+[pre+'_'+feat for pre in ['pleth', 'ecgii'] for feat in ['mean', 'sd', 'cv', 'mad', 'n', 'min', 'max', 'median', 'range', 'range_ratio']]))

,filename,series,left,right,label,hr_mean,hr_sd,hr_cv,hr_mad,hr_n,hr_min,hr_max,hr_median,hr_range,hr_range_ratio,hr_data_den,hr_data_den_trail,hr_data_den_trail2,hr_delta_t,hr_max_gap,hr_delta_mean,hr_delta_sd,hr_MWW_stat,hr_MWW_pvalue,hr_KS_stat,hr_KS_pvalue,hr_t_stat,hr_t_pvalue,hr_F_stat,hr_F_pvalue,hr_slope,hr_rslope,hr_slope_before,hr_slope_after,hr_num_breakpoint,hr_diff1_max,hr_diff1_min,hr_max_grad,hr_min_grad,hr_quad_rsq,hr_quad_coef1,hr_quad_coef2,hr_quad_resvar,hr_osi_up,hr_osi_down,hr_osi_ratio,rr_mean,rr_sd,rr_cv,rr_mad,rr_n,rr_min,rr_max,rr_median,rr_range,rr_range_ratio,rr_data_den,rr_data_den_trail,rr_data_den_trail2,rr_delta_t,rr_max_gap,rr_delta_mean,rr_delta_sd,rr_MWW_stat,rr_MWW_pvalue,rr_KS_stat,rr_KS_pvalue,rr_t_stat,rr_t_pvalue,rr_F_stat,rr_F_pvalue,rr_slope,rr_rslope,rr_slope_before,rr_slope_after,rr_num_breakpoint,rr_diff1_max,rr_diff1_min,rr_max_grad,rr_min_grad,rr_quad_rsq,rr_quad_coef1,rr_quad_coef2,rr_quad_resvar,rr_osi_up,rr_osi_down,rr_osi_ratio,spo2_mean,spo2_sd,spo2_cv,spo2_mad,spo2_n,spo2_min,spo2_max,spo2_median,spo2_range,spo2_range_ratio,spo2_data_den,spo2_data_den_trail,spo2_data_den_trail2,spo2_delta_t,spo2_max_gap,spo2_delta_mean,spo2_delta_sd,spo2_MWW_stat,spo2_MWW_pvalue,spo2_KS_stat,spo2_KS_pvalue,spo2_t_stat,spo2_t_pvalue,spo2_F_stat,spo2_F_pvalue,spo2_slope,spo2_rslope,spo2_slope_before,spo2_slope_after,spo2_num_breakpoint,spo2_diff1_max,spo2_diff1_min,spo2_max_grad,spo2_min_grad,spo2_quad_rsq,spo2_quad_coef1,spo2_quad_coef2,spo2_quad_resvar,spo2_osi_up,spo2_osi_down,spo2_osi_ratio,pleth_skew,pleth_kurtosis,pleth_entropy,ecgii_snr,hrrr_ratio_mean,hrrr_ratio_sd,hrrr_ratio_cv,hrrr_ratio_mad,hrrr_ratio_n,hrrr_ratio_min,hrrr_ratio_max,hrrr_ratio_median,hrrr_ratio_range,hrrr_ratio_range_ratio
0,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1515595508,1515596030,1,139.865248,3.468676,0.024800,1.134752,141.0,126.0,146.0,141.0,20.0,0.141844,0.270115,0.782809,0.781111,2.048,2.048,52.464179,0.724842,0.0,1.161034e-54,1.000000,0.000000e+00,120.650057,1.440056e-272,0.684180,1.830132e-02,-0.011264,-0.016833,-0.068257,-0.066920,1.0,9.765626,-1.953125,6.616793,-1.135976,0.028826,1.934979e+03,-0.011264,11.684885,0.0,0.0,NaN,16.865248,4.144221,0.245725,-0.865248,141.0,12.0,30.0,16.0,18.0,1.125000,0.270115,0.782809,0.781111,2.048,2.048,1.501612,1.345150,10750.5,1.971148e-03,0.222589,5.499935e-04,3.911740,1.115378e-04,2.192088,6.087411e-07,-0.059093,-0.051371,-0.204129,-0.020814,1.0,5.859375,-2.929688,1.621756,-0.939216,0.555763,9.434094e+03,-0.059093,7.629583,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.909494,8.677217,1.638820,0.188865,0.197783,141.0,4.800000,11.916667,8.875000,7.116667,0.801878
1,20190523_1053954_1173378.h5,/data/numerics/HR.HR:value,1515638422,1515643674,1,138.085106,3.485663,0.025243,-0.085106,141.0,117.0,144.0,138.0,27.0,0.195652,0.026847,0.776544,0.586667,1.024,2.048,56.192058,0.294731,0.0,6.993467e-55,1.000000,0.000000e+00,151.718107,1.944322e-304,1.193262,2.600455e-01,-0.012794,-0.018803,-0.021840,0.049788,1.0,19.531247,-9.765626,5.024068,-2.704814,0.036836,2.726082e+03,-0.012794,11.702299,1.0,0.0,1.0,18.971631,3.253445,0.171490,-0.971631,141.0,13.0,27.0,18.0,14.0,0.777778,0.026847,0.774769,0.584444,1.024,2.048,1.607995,0.367932,10009.5,8.864709e-05,0.183790,7.337787e-03,4.156996,4.126091e-05,0.807123,1.813444e-01,-0.029295,-0.032894,-0.111594,-0.087204,1.0,2.441406,-2.929688,1.371091,-1.531050,0.221674,5.944728e+03,-0.029295,8.238511,0.0,0.0,NaN,88.338636,4.005962,0.045348,1.211364,132.0,79.9,93.6,89.55,13.7,0.152987,0.025133,0.777834,0.556667,1.024,7.168,3.750668,1.757404,5782.0,3.047274e-16,0.411765,2.802647e-12,-10.649352,7.668429e-23,3.173989,5.937473e-13,-0.037619,-3.764631e-02,-7.535343e-02,-1.351417e-01,1.0,5.566405,-5.371093,3.133064e+00,-3.063096e+00,0.240654,7697.806195,-3.761896e-02,1.218578e+01,0.0,0.0,NaN,-859

In [85]:
### Export to model-specific datasets

suf = '_try2'

ds = combined.drop(columns=[pre+'_'+feat for pre in ['bps', 'bpd'] for feat in featuresList]+[pre+'_'+feat for pre in ['pleth', 'ecgii'] for feat in ['mean', 'sd', 'cv', 'mad', 'n', 'min', 'max', 'median', 'range', 'range_ratio']])

ds.to_csv(f'data_full{suf}.csv', index=False)
ds.drop(columns=['series', 'left', 'right']).to_csv(f'data_withfn{suf}.csv', index=False)

ds[ds['series'] == '/data/numerics/HR.HR:value'].drop(columns=['series', 'left', 'right']).to_csv(f'data_hr_withfn{suf}.csv', index=False)
ds[ds['series'] == '/data/numerics/RR.RR:value'].drop(columns=['series', 'left', 'right']).to_csv(f'data_rr_withfn{suf}.csv', index=False)
ds[ds['series'].isin(['/data/numerics/SpO₂.SpO₂:value', '/data/numerics/SpO₂T.SpO₂T:value'])].drop(columns=['series', 'left', 'right']).to_csv(f'data_spo2_withfn{suf}.csv', index=False)